In [2]:
import xgboost

In [3]:
import os

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import numpy

In [7]:
import pandas

In [8]:
import datetime

In [9]:
import cPickle as pickle

In [10]:
import cPickle

In [11]:
import itertools

In [12]:
import matplotlib.pyplot as plt

import memory_profiler
import line_profiler
%load_ext memory_profiler

In [13]:
method = 'logistic regression'
#method = 'random forest'

In [14]:
logic_C = 1e5

In [15]:
rf_n_estimators = 10
rf_max_depth = 3

In [16]:
random_seed = 0;

In [17]:
n_uesr_clusters = 1

In [18]:
bMonitorAllTrees = False
bTreeDump = True

In [19]:
bFullData = True
bCV = False
bLeaveOneOut = True
bTwoModel = False
bOnlyTwoWeekTrain = False

In [20]:
bParallel = False;
if bParallel:
    core_num = 4;
else:
    core_num = 1

In [21]:
os.chdir('C:\Bimbo')

In [22]:
pd.set_option('display.max_columns',None);

In [23]:

test = pd.read_csv("test.csv", dtype = {
        'id':np.int32,
        'Semana':np.int8,
        'Agencia_ID':np.int16,
        'Canal_ID':np.int8, 
        'Ruta_SAK':np.int16,
        'Cliente_ID':np.int32,
        'Producto_ID':np.int32,
    })

In [24]:

train = pd.read_csv(
    "train.csv", 
    usecols = ['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil','Venta_uni_hoy','Venta_hoy'],
    dtype = {
        'Semana':np.int8,
        'Agencia_ID':np.int16,
        'Canal_ID':np.int8, 
        'Ruta_SAK':np.int16,
        'Cliente_ID':np.int32,
        'Producto_ID':np.int32,
        'Venta_uni_hoy':np.int32,
        'Venta_hoy':np.float32,
        #'Dev_uni_proxima':np.int32,
        #'Dev_proxima':np.float32,
        'Demanda_uni_equil':np.float32
    })

In [25]:
train.shape

(74180464, 9)

In [26]:
test.shape

(6999251, 7)

In [27]:
train.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Demanda_uni_equil'],
      dtype='object')

In [28]:
test.columns

Index([u'id', u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK',
       u'Cliente_ID', u'Producto_ID'],
      dtype='object')

In [29]:
train.dtypes

Semana                  int8
Agencia_ID             int16
Canal_ID                int8
Ruta_SAK               int16
Cliente_ID             int32
Producto_ID            int32
Venta_uni_hoy          int32
Venta_hoy            float32
Demanda_uni_equil    float32
dtype: object

In [30]:
train['log_demand']=np.log1p(train['Demanda_uni_equil'])

In [31]:
train.memory_usage().sum()

2225413992L

In [32]:
test.memory_usage().sum()

125986590L

In [33]:
if bFullData:
    t1 = train
    t2 = test
else:
    if bCV:
        t1 = train.loc[train['Semana'] <= 5,:].copy()
        t2 = train.loc[(train['Semana'] ==6) | (train['Semana'] ==7),:].copy()
        t3 = train.loc[(train['Semana'] ==8) | (train['Semana'] ==9),:].copy()
    else:
        t1 = train.loc[train['Semana'] <= 7,:].copy()
        #t1 = train.loc[(train['Semana'] == 6) | (train['Semana'] == 7),:].copy()
        t2 = train.loc[(train['Semana'] == 8) | (train['Semana'] == 9),:].copy()
    del test
    del train

In [34]:
#t1.drop(t1.index[t1['Demanda_uni_equil']==0.0], inplace =True)

In [35]:

mean_total = np.mean(t1['log_demand'])

In [36]:
short_hand_dict = {'A':'Agencia_ID','Ca':'Canal_ID','R':'Ruta_SAK','C':'Cliente_ID','P':'Producto_ID'}

In [37]:
all_terms = [['P','C','A','Ca','R'],
             ['A','Ca','R','C'],
             ['A','Ca','R','P'],
             ['A','Ca','P'],
             #['Ca','R','P'],
             ['A','R','P'],
             #['Ca','R','C'],
             ['A','R','C'],
             #['A','Ca','C'],
             ['A','Ca','R'],
             ['C','P'],
             ['C'],
             ['P'],
             ['A'],
             ['R'],
             ['Ca'],
             ['P','R'],
             ['P','C','A']]

In [38]:
all_terms_joined = map(str.join,itertools.repeat('',len(all_terms)),all_terms)

In [39]:
match_cols_dict = {all_terms_joined[i]:[short_hand_dict[key] for key in all_terms[i]] 
                  for i in xrange(len(all_terms))}

In [40]:
import gc

In [41]:
if bFullData:
    t2['log_demand'] = [0 for i in xrange(len(t2))]

In [42]:
%%time
for key in match_cols_dict:
    print key
    mean_groupby = t1[match_cols_dict[key] +['log_demand']].groupby(match_cols_dict[key]).mean()
    if bLeaveOneOut:
        sum_groupby = t1[match_cols_dict[key] +['log_demand']].groupby(match_cols_dict[key]).sum()
        sum_groupby.columns = ['temp_sum']
        size_groupby = t1[match_cols_dict[key] +['log_demand']].groupby(match_cols_dict[key]).size()
        size_groupby.name = 'temp_size'
        t1 = t1.merge(sum_groupby.join(size_groupby), how='left', left_on = match_cols_dict[key], 
                 right_index=True, suffixes=('','_' + key))
        t1['log_demand_' + key] = (t1['temp_sum'] - t1['log_demand']) / (t1['temp_size'] - 1)

        t1.drop(['temp_sum','temp_size'],axis = 1, inplace = True)
    else:
        t1 = t1.merge(mean_groupby, how='left', left_on = match_cols_dict[key], 
                 right_index=True, suffixes=('','_' + key))    
    t2 = t2.merge(mean_groupby, how='left', left_on = match_cols_dict[key], 
             right_index=True, suffixes=('','_' + key))
    if bCV:
        t3 = t3.merge(mean_groupby, how='left', left_on = match_cols_dict[key], 
                 right_index=True, suffixes=('','_' + key))
    if bLeaveOneOut:
        del sum_groupby
        del size_groupby
    del mean_groupby

    gc.collect()


ARP
A
C
PCA
ACaRP
Ca
PR
P
ARC
PCACaR
ACaRC
ACaP
R
CP
ACaR
Wall time: 30min 2s


In [43]:
t1.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Demanda_uni_equil',
       u'log_demand', u'log_demand_ARP', u'log_demand_A', u'log_demand_C',
       u'log_demand_PCA', u'log_demand_ACaRP', u'log_demand_Ca',
       u'log_demand_PR', u'log_demand_P', u'log_demand_ARC',
       u'log_demand_PCACaR', u'log_demand_ACaRC', u'log_demand_ACaP',
       u'log_demand_R', u'log_demand_CP', u'log_demand_ACaR'],
      dtype='object')

In [44]:
t1_merge = t1
t2_merge = t2
if bCV:
    t3_merge = t3
    del t3
del t1
del t2

In [45]:
store_size = t1_merge[['Cliente_ID','Venta_hoy']].groupby(['Cliente_ID']).sum()
store_size.columns = ['store_size']

In [46]:
t1_merge = t1_merge.merge(store_size, how='left', left_on = 'Cliente_ID', 
             right_index=True)
t2_merge = t2_merge.merge(store_size, how='left', left_on = 'Cliente_ID', 
             right_index=True)
if bCV:
    t3_merge = t3_merge.merge(store_size, how='left', left_on = 'Cliente_ID', 
             right_index=True)

In [47]:
t1_merge['log_demand_CP_from_store_size'] = np.log1p(np.expm1(t1_merge['log_demand_P']) * t1_merge['store_size'])

In [48]:
t2_merge['log_demand_CP_from_store_size'] = np.log1p(np.expm1(t2_merge['log_demand_P']) * t2_merge['store_size'])

t1['price'] = t1['Venta_hoy'] / t1['Venta_uni_hoy']

price_CP = t1[['Cliente_ID','Producto_ID','price']].groupby(['Producto_ID','Cliente_ID'],as_index = False).mean()

mean_price = t1[['Producto_ID','price']].groupby(['Producto_ID']).mean()

mean_price.columns = ['mean_price']

price_CP = price_CP.merge(mean_price, how = 'left', left_on = 'Producto_ID', right_index = True)

price_CP['relative_price'] = price_CP['price'] / price_CP['mean_price']

price_CP['relative_price'].max()

price_CP['relative_price'].min()

plt.hist(price_CP.loc[price_CP['relative_price'].notnull(),'relative_price'], bins = 30)

plt.show()

t1['Cliente_ID'].nunique()

t1.drop('store_size', inplace = True)
t2.drop('store_size', inplace = True)

In [49]:
t1_merge.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Demanda_uni_equil',
       u'log_demand', u'log_demand_ARP', u'log_demand_A', u'log_demand_C',
       u'log_demand_PCA', u'log_demand_ACaRP', u'log_demand_Ca',
       u'log_demand_PR', u'log_demand_P', u'log_demand_ARC',
       u'log_demand_PCACaR', u'log_demand_ACaRC', u'log_demand_ACaP',
       u'log_demand_R', u'log_demand_CP', u'log_demand_ACaR', u'store_size',
       u'log_demand_CP_from_store_size'],
      dtype='object')

In [50]:
t2_merge.columns

Index([u'id', u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK',
       u'Cliente_ID', u'Producto_ID', u'log_demand', u'log_demand_ARP',
       u'log_demand_A', u'log_demand_C', u'log_demand_PCA',
       u'log_demand_ACaRP', u'log_demand_Ca', u'log_demand_PR',
       u'log_demand_P', u'log_demand_ARC', u'log_demand_PCACaR',
       u'log_demand_ACaRC', u'log_demand_ACaP', u'log_demand_R',
       u'log_demand_CP', u'log_demand_ACaR', u'store_size',
       u'log_demand_CP_from_store_size'],
      dtype='object')

In [51]:
len(t1_merge)

74180464

In [52]:
len(t2_merge)

6999251

In [53]:
if bCV:
    print len(t3_merge)

In [54]:
len(t1_merge)

74180464

In [55]:
len(t2_merge)

6999251

In [56]:
%%time
lag_to = 4
for i in range(1,lag_to + 1):
    print i
    t1_lag1_for_merge = t1_merge[match_cols_dict['CP'] + ['Semana','log_demand']].copy()
    t1_lag1_for_merge['Semana'] += i
    t1_lag1_for_merge = t1_lag1_for_merge.groupby(match_cols_dict['CP'] + ['Semana'], as_index = False).mean()
    t1_merge = t1_merge.merge(t1_lag1_for_merge, how='left', on = match_cols_dict['CP'] + ['Semana'],suffixes=('','_lag' + ("%i" % i)))
    t2_merge = t2_merge.merge(t1_lag1_for_merge, how='left', on = match_cols_dict['CP'] + ['Semana'],suffixes=('','_lag' + ("%i" % i)))
    del t1_lag1_for_merge
    gc.collect()

1
2
3
4
Wall time: 16min 49s


In [57]:
t1_merge.columns

Index([u'Semana', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK', u'Cliente_ID',
       u'Producto_ID', u'Venta_uni_hoy', u'Venta_hoy', u'Demanda_uni_equil',
       u'log_demand', u'log_demand_ARP', u'log_demand_A', u'log_demand_C',
       u'log_demand_PCA', u'log_demand_ACaRP', u'log_demand_Ca',
       u'log_demand_PR', u'log_demand_P', u'log_demand_ARC',
       u'log_demand_PCACaR', u'log_demand_ACaRC', u'log_demand_ACaP',
       u'log_demand_R', u'log_demand_CP', u'log_demand_ACaR', u'store_size',
       u'log_demand_CP_from_store_size', u'log_demand_lag1',
       u'log_demand_lag2', u'log_demand_lag3', u'log_demand_lag4'],
      dtype='object')

In [58]:
gc.collect()

0

In [59]:
len(t1_merge)

74180464

In [60]:
len(t2_merge)

6999251

In [61]:
if bCV:
    t3_merge = t3_merge.merge(t1_lag1_for_merge, how='left', on=match_cols_dict['CP'] + ['Semana'],suffixes=('','_lag1_CP'))

In [62]:
#t2_merge.loc[(t2_merge['Cliente_ID'] == 26) & (t2_merge['Producto_ID'] == 31393),match_cols_CP +['Semana','log_demand_lag1_CP','log_demand']]

In [63]:
#t1_merge.sort_values(match_cols_CP +['Semana','log_demand_lag1_CP','log_demand'])[match_cols_CP +['Semana','log_demand_lag1_CP','log_demand']]

In [64]:
match_cols = ['Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID']

In [65]:
import math
import ml_metrics

In [66]:
#use hexbin to plot point density of million of point that are not possible to visualize through ordinary scatter plot
def DensePlot(x,y,xlim,ylim):
    #x,y should be panda dataframes
    #plt.figure(figsize=(27, 27))
    ax = plt.gca()
    sel_rows = (x >= xlim[0]) & (x <= xlim[1]) & (y >= ylim[0]) & (y <= ylim[1])
    ax.hexbin(x[sel_rows.values],y[sel_rows.values],cmap=plt.cm.hot,bins = 'log')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    plt.show()

In [67]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import patsy
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

In [85]:
#initialize pred column

t2_merge['pred'] = np.nan
current_na = [True for i in xrange(len(t2_merge))]
if bCV:
    t3_merge['pred'] = np.nan
    current_na_t3 = [True for i in xrange(len(t3_merge))]
else:
    t3_merge=None
    current_na_t3 = None

In [77]:
def FillPreds(terms_short, # short hand for terms, e.g. ['C','ACaRC','ACaR','ARP','P','R','CP']
              t1_merge, 
              t2_merge, 
              t3_merge,
              current_na, # record the rows whose prediction value remains to be filled
              current_na_t3, # record the rows whose prediction value remains to be filled
              additional_terms_in_formula, # can add in quadratic terms and interaction terms. e.g. '+ I(log_demand_CP**2) + log_demand_CP:ACaR'
              alpha,  # strength of regularization
              L1_wt,  # weight of L1 norm regularization term, if 0.0 then only L2 norm is used, if 1.0 then only L1 norm is used
              bParamTuning, # Parameter tuning(experiments) or officially write in prediction?  If False, then the predictions writen to pred column will not be changed. If True, then it gernates the optimization result for evaluation purpose but a later function call can still overwrite the prediction writen in this call. 
              bFullData, 
              bPrintFitSum): # can turn off the fit summary table by setting this to false.
    terms = ['log_demand_' + str for str in terms_short]
    myFormula = 'log_demand~' + '+'.join(terms) + additional_terms_in_formula
    
    #below two line are extracting the rows that does not have any NA in the given terms.
    train_no_NA_rows = t1_merge[terms].notnull().all(axis=1)
    no_NA_rows = t2_merge[terms].notnull().all(axis=1)
    if t3_merge is not None:
        no_NA_rows_t3 = t3_merge[terms].notnull().all(axis=1)
    
    y, X = patsy.dmatrices(myFormula, t1_merge.loc[train_no_NA_rows,terms + ['log_demand']], return_type='dataframe')
    #mod = smf.ols(formula = myFormula , data = t1_merge[train_no_NA_rows])
    #ss = StandardScaler(copy = False)
    #ss.fit_transform(X.iloc[:,1:])
    #ss.fit_transform(X);
    #X['Intercept'] = np.float32(1.0)
    mod = sm.OLS(y , X)
    init = np.linalg.solve(X.values.transpose().dot(X.values) + alpha * len(X) * np.identity(X.shape[1]),X.values.transpose().dot(y.iloc[:,0].values))
    del X
    gc.collect()
    #init = mod.fit()
    res = mod.fit_regularized(cnvrg_tol=1e-04, alpha = alpha , L1_wt = L1_wt, start_params = init)


    #res = mod.fit()

    if bPrintFitSum:
        print res.summary()
    # fill in rows that are currently na in pred but none na in the given terms in both train and test
    to_fill_rows = no_NA_rows & current_na   
    test_matrix = patsy.dmatrix('+'.join(terms) + additional_terms_in_formula, t2_merge.loc[to_fill_rows,terms], return_type='dataframe')
    #ss.transform(test_matrix.iloc[:,1:])
    t2_merge.loc[to_fill_rows,'pred'] = np.expm1(res.predict(test_matrix))
    del test_matrix
    gc.collect()
    if t3_merge is not None:
        to_fill_rows_t3 = no_NA_rows_t3 & current_na_t3
        test_matrix = patsy.dmatrix('+'.join(terms) + additional_terms_in_formula, t3_merge.loc[to_fill_rows_t3,terms], return_type='dataframe')
        t3_merge.loc[to_fill_rows_t3,'pred'] = np.expm1(res.predict(test_matrix))
        del test_matrix
        gc.collect()
    print ' + '.join(terms_short)
    
    train_score = ml_metrics.rmsle(np.expm1(res.fittedvalues),np.expm1(y.iloc[:,0]))
    
    if not bFullData:
        if t3_merge is not None:
            print "cv new fill %d, test new fill %d, train score is %f, cv_score is %f, test score of the new fill is %f" % (to_fill_rows.sum(),                                                            
                                                            to_fill_rows_t3.sum(),
                                                            train_score,
                                                            ml_metrics.rmsle(t2_merge.loc[to_fill_rows,'pred'],t2_merge.loc[to_fill_rows,'Demanda_uni_equil']),
                                                            ml_metrics.rmsle(t3_merge.loc[to_fill_rows_t3,'pred'],t3_merge.loc[to_fill_rows_t3,'Demanda_uni_equil']))
        else:
            print "new fill %d, train score is %f, test score of the new fill is %f" % (to_fill_rows.sum(),                                                            
                                                            train_score,
                                                            ml_metrics.rmsle(t2_merge.loc[to_fill_rows,'pred'],t2_merge.loc[to_fill_rows,'Demanda_uni_equil']))
    else:
        print "new fill %d, train score is %f" % (to_fill_rows.sum(),
                                                  #ml_metrics.rmsle(res.fittedvalues,t1_merge.loc[t1_merge[terms].notnull().all(axis=1), 'Demanda_uni_equil']))
                                                  train_score)
    if not bParamTuning:
        # to integrate the prediction to the pred column
        current_na &= (-to_fill_rows)
        if t3_merge is not None:
            current_na_t3 &= (-to_fill_rows_t3)
    return current_na, current_na_t3

%%time
current_na, current_na_t3 = FillPreds(['C','ACaRC','ACaR','ARP','P','R','CP'],t1_merge,t2_merge,t3_merge,current_na,current_na_t3,
                       '',
                       0.1,0.0,True, bFullData, True)

pred_regularized = t2_merge['pred'].copy()

%%time
current_na, current_na_t3 = FillPreds(['C','ACaRC','ACaR','ARP','P','R','CP'],t1_merge,t2_merge,t3_merge,current_na,current_na_t3,
                       '',
                       0.0,0.0,True, bFullData, True)

pred_plain = t2_merge['pred'].copy()

import itertools

log_error1 = pd.Series(list(itertools.starmap(ml_metrics.msle,zip(t2_merge['Demanda_uni_equil'],pred_regularized))))


log_error2 = pd.Series(list(itertools.starmap(ml_metrics.msle,zip(t2_merge['Demanda_uni_equil'],pred_plain))))

better = log_error1 < log_error2

worse = log_error1 >= (log_error2 +0.16)

pred_combine = pd.concat([pred_regularized[better],pred_plain[worse]])

pred_combine.index





ml_metrics.rmsle(t2_merge.loc[pred_combine.index,'Demanda_uni_equil'], pred_combine)

t2_merge['plain_better'] = np.float32(worse)

better.sum()

worse.sum()

temp = pd.DataFrame.join(t2_merge[['plain_better','Cliente_ID','Producto_ID']].groupby(['Cliente_ID','Producto_ID']).sum(),
               t2_merge[['plain_better','Cliente_ID','Producto_ID']].groupby(['Cliente_ID','Producto_ID']).size().rename('size'))

temp['all_plain_better']=(temp['plain_better']==temp['size'])

temp['all_plain_better'].sum()

len(temp['all_plain_better'])

temp['plain_better'].sum()

#temp.columns = ['Cliente_ID','Producto_ID','plain_better']

file_name='plain_better_margin_0.16'
if bCV:
    pickle.dump(temp,open(file_name + '_cv.sav','wb'))
else:
    pickle.dump(temp,open(file_name + '.sav','wb'))

DensePlot(pred_regularized[better],pred_plain[better],[0,200],[0,200])

DensePlot(pred_regularized[worse],pred_plain[worse],[0,200],[0,200])

DensePlot(t2_merge.loc[better.values,'Demanda_uni_equil'], pred_regularized.loc[better.values],[0,200],[0,200])

DensePlot(t2_merge.loc[worse.values,'Demanda_uni_equil'], pred_regularized.loc[worse.values],[0,200],[0,200])

DensePlot(t2_merge.loc[better.values,'Demanda_uni_equil'], pred_plain.loc[better.values],[0,200],[0,200])

DensePlot(t2_merge.loc[worse.values,'Demanda_uni_equil'], pred_plain.loc[worse.values],[0,200],[0,200])

temp1 = pickle.load(open('plain_better_0.2_margin.sav','rb'))

temp2 = pickle.load(open('plain_better.sav','rb'))

(temp1 - temp2).sum()

if bOnlyTwoWeekTrain:
    if bFullData:
        t1_merge = t1_merge.loc[(t1_merge['Semana'] == 8) | (t1_merge['Semana'] == 9),:]
    else:
        t1_merge = t1_merge.loc[(t1_merge['Semana'] == 6) | (t1_merge['Semana'] == 7),:]

In [78]:
import sys

In [80]:
def XGBGridSearchAllData(t1_merge, t2_merge, current_na,predictor, default_params, new_params, bFullData, bParamTuning):
    xgb_reg = xgboost.XGBRegressor(**default_params)
    min_test_score = 10000;
    dict_list = []
    if new_params != {}:
        for i in xrange(len(new_params.values()[0])):
            dict_list.append({key:value[i] for key,value in new_params.iteritems()})
    else:
        dict_list.append(default_params)
    print dict_list
    for one_new_param in dict_list:
        xgb_reg.set_params(**one_new_param)
        if bFullData:
            xgb_reg.fit(t1_merge[predictor],
                    t1_merge['log_demand'],
                    [(t1_merge[predictor], t1_merge['log_demand'])],
                    eval_metric  = 'rmse')
            if "log_demand_lag1" in predictor:
                print "has lag1!"
                t2_merge.loc[t2_merge['Semana']==8,'pred'] = xgb_reg.predict(t2_merge.loc[t2_merge['Semana']==8,predictor])
                lag1_for_merge = t2_merge.loc[t2_merge['Semana']==8,match_cols_dict['PCACaR'] + ['Semana' + 'pred']]
                lag1_for_merge['Semana'] += 1
                
        else:
            xgb_reg.fit(t1_merge[predictor],
                    t1_merge['log_demand'],
                    [(t1_merge[predictor], t1_merge['log_demand']), 
                     (t2_merge[predictor], t2_merge['log_demand'])],
                    eval_metric  = 'rmse')
        if not bFullData:
            test_score = ml_metrics.rmse(t2_merge['log_demand'],xgb_reg.predict(t2_merge[predictor]))
        else:
            test_score = 0
        train_score = ml_metrics.rmse(t1_merge['log_demand'],xgb_reg.predict(t1_merge[predictor]))
        print "parameter: [%s], value = [%s], test score: %f, train score: %f" % (
            ','.join(one_new_param.keys()),
            ','.join(["%f" % val for val in one_new_param.values()]),
            test_score,
            train_score
        )
        sys.stdout.flush()
        if (test_score < min_test_score):
            min_test_score = test_score
            min_xgb_reg = xgb_reg
    if not bParamTuning:
        t2_merge['pred'] = np.expm1(min_xgb_reg.predict(t2_merge[predictor]))
    
    return min_xgb_reg

In [81]:
default_params = {

    'objective':'reg:linear', 
    'nthread':-1, 
    'gamma':0,
    'min_child_weight':1, 
    'max_delta_step':0,  
    'colsample_bytree':0.6,  
    'reg_alpha':0, 
    'reg_lambda':1, 
    'scale_pos_weight':1, 
    'seed':0, 
    'missing':None,
    #'objective':'reg:linear', 
    'silent':0,
    'subsample':0.8,
    'max_depth':10,
    'learning_rate':0.1,
    'n_estimators':30
}

In [82]:
%%time
predictor = ['log_demand_' + str for str in ['C','ACaRC','ACaR','ARP','P','A','R','CP','CP_from_store_size','lag2','lag3','lag4']]
min_xgb_reg = XGBGridSearchAllData(t1_merge.loc[(t1_merge['Semana'] == 9),:], t2_merge,current_na, predictor, default_params, {
        'n_estimators':[440],
        'learning_rate':[0.01],
        'subsample':[0.7],
        'colsample_bytree':[1.0],
        'max_depth':[6]
    }, bFullData, False)

[{'n_estimators': 440, 'subsample': 0.7, 'learning_rate': 0.01, 'colsample_bytree': 1.0, 'max_depth': 6}]
[0]	validation_0-rmse:1.36711
[1]	validation_0-rmse:1.35506
[2]	validation_0-rmse:1.34315
[3]	validation_0-rmse:1.33137
[4]	validation_0-rmse:1.31972
[5]	validation_0-rmse:1.3082
[6]	validation_0-rmse:1.29681
[7]	validation_0-rmse:1.28555
[8]	validation_0-rmse:1.27441
[9]	validation_0-rmse:1.2634
[10]	validation_0-rmse:1.25252
[11]	validation_0-rmse:1.24176
[12]	validation_0-rmse:1.23112
[13]	validation_0-rmse:1.2206
[14]	validation_0-rmse:1.2102
[15]	validation_0-rmse:1.19992
[16]	validation_0-rmse:1.18975
[17]	validation_0-rmse:1.1797
[18]	validation_0-rmse:1.16976
[19]	validation_0-rmse:1.15994
[20]	validation_0-rmse:1.15023
[21]	validation_0-rmse:1.14063
[22]	validation_0-rmse:1.13114
[23]	validation_0-rmse:1.12176
[24]	validation_0-rmse:1.1125
[25]	validation_0-rmse:1.10334
[26]	validation_0-rmse:1.09428
[27]	validation_0-rmse:1.08534
[28]	validation_0-rmse:1.0765
[29]	validat

In [83]:
if bFullData:
    #write_frame = ["{0},{1}".format(t2["id"][i], t2_merge['pred']) for i in xrange(len(t2))]
    #write_frame = ["id,Demanda_uni_equil"] + write_frame
    #with open("predictions.csv", "w+") as f:
    #    f.write("\n".join(write_frame))
    towrite = t2_merge[['id', 'pred']]
    towrite.columns = ['id','Demanda_uni_equil']
    towrite.sort_values(['id'], ascending=[True]).to_csv("prediction_xgboost.csv", index=False)
else:
    plt.figure(figsize=(27, 27))
    xmax = t2_merge['Demanda_uni_equil'].max()
    ax = plt.subplot(1, 2, 1)
    sel_rows = ((t2_merge['Demanda_uni_equil'] < 5000) & (preds_df < 5000))
    ax.hexbin(t2_merge.loc[sel_rows.values,'Demanda_uni_equil'],preds_df.loc[sel_rows.values],cmap=plt.cm.hot,bins = 'log')
    ax.set_xlabel('actual')
    ax.set_ylabel('pred')
    ax.set_xlim([0,xmax])
    ax.set_ylim([0,xmax])
    ax = plt.subplot(1, 2, 2)
    sel_rows = ((t2_merge['Demanda_uni_equil'] < 200) & (preds_df < 200))
    ax.hexbin(t2_merge.loc[sel_rows.values,'Demanda_uni_equil'],preds_df.loc[sel_rows.values],cmap=plt.cm.hot,bins = 'log')
    ax.set_xlim([0, 200])
    ax.set_ylim([0, 200])
    ax.set_xlabel('actual')
    ax.set_ylabel('pred')
    plt.figure(figsize=(27, 27))
    ax = plt.subplot(1, 2, 1)
    sel_rows = ((t2_merge['Demanda_uni_equil'] < 50) & (preds_df < 50))
    ax.hexbin(t2_merge.loc[sel_rows.values,'Demanda_uni_equil'],preds_df.loc[sel_rows.values],cmap=plt.cm.hot,bins = 'log',gridsize=(50,50))
    ax.set_xlim([0, 50])
    ax.set_ylim([0, 50])
    ax.set_xlabel('actual')
    ax.set_ylabel('pred')


In [84]:
t2_pred_xgboost = t2_merge['pred']